In [1]:
import pandas as pd
import seaborn as sns

import matplotlib.pyplot as plt

In [2]:
## load cleaned data
segment_revenue_and_count = pd.read_pickle("segment_revenue_and_count.pickle")
segment_count = pd.read_pickle("segment_count.pickle")

In [3]:
segment_count.sort_values("count", ascending = False)

,Age,Gender,Income,Children,Housing,Relationship,count
636,Unknown,Unknown,Unknown,Unknown,Unknown,Unknown,64538
634,Unknown,Female,Unknown,Unknown,Unknown,Unknown,34395
734,Unknown,Female,Middle,None,House,Married,22247
1438,65+,Female,Low,None,House,Married,19944
2239,Unknown,Female,Unknown,Unknown,House,Unknown,18974
...,...,...,...,...,...,...,...
3450,65+,Male,High,Baby,House,Single,1
3463,55-64,Unknown,VeryHigh,Youth,Flat,Unknown,1
3480,18-24,Male,VeryHigh,None,Flat,Married,1
3481,18-24,Male,Middle,None,Unknown,Unknown,1


In [4]:
segment_revenue_and_count.head()

,Age,Gender,Income,Children,Housing,Relationship,PurchaseCategoryA,PurchaseCategoryB,revenue,total_purchases
0,35-44,Male,Middle,Youth,Flat,Married,Home,Beds & Mattresses,733.450,6
1,Unknown,Female,Low,Youth,House,Unknown,Home,Beds & Mattresses,10745.972,67
2,45-54,Unknown,VeryHigh,None,House,Separated,Home,Beds & Mattresses,2204.830,16
3,25-34,Unknown,Middle,Teenager,House,Married,Home,Beds & Mattresses,25627.180,145
4,35-44,Unknown,Middle,None,Flat,Unknown,Home,Beds & Mattresses,3405.240,21


In [5]:
## avg revenue per purchase
segment_revenue_and_count["avg_spend"] = segment_revenue_and_count.revenue/segment_revenue_and_count.total_purchases

In [6]:
segment_revenue_and_count.head()

,Age,Gender,Income,Children,Housing,Relationship,PurchaseCategoryA,PurchaseCategoryB,revenue,total_purchases,avg_spend
0,35-44,Male,Middle,Youth,Flat,Married,Home,Beds & Mattresses,733.450,6,122.241667
1,Unknown,Female,Low,Youth,House,Unknown,Home,Beds & Mattresses,10745.972,67,160.387642
2,45-54,Unknown,VeryHigh,None,House,Separated,Home,Beds & Mattresses,2204.830,16,137.801875
3,25-34,Unknown,Middle,Teenager,House,Married,Home,Beds & Mattresses,25627.180,145,176.739172
4,35-44,Unknown,Middle,None,Flat,Unknown,Home,Beds & Mattresses,3405.240,21,162.154286


In [7]:
def get_spend_stats_by_col(spend_df,
                           col_name, 
                           filter_col = None, 
                           filter_value = None, 
                           remove_unknowns = False, 
                           include_segment_count = False,
                           sort_order = ["revenue","avg_spend"]):
    
    """
    This function takes the spend data and generates revenue and total purchase stats based on a field
    We determine:
        1. Grouped total revenue for categories in that field
        2. Grouped total purchases for categories in that field
        3. The percentage share of revenue for a group
        4. The percentage share of total purchases for a group
        5. The total number (and %) of customers in a group
        6. The revenue generated per customers in that group
    
    Parameters:
        spend_df: spend data (segment_revenue_and_count)
        col_name: the column to generate stats on (eg.: Age, Income, Gender etc.)
        filter_col and filter_value: To filter the spend data on a purchase subc4ategory and generated some fine grained analysis
        remove_unknowns: Boolean to consider unknowns in the dataset or not
        include_segment_count: Boolean to include segment counts from segment_count dataframe
        sort_order: return the dataframe sorted in some specific order
    
    """
    
    if filter_col:
        spend_df = spend_df[spend_df[filter_col] == filter_value]
        
    if remove_unknowns:
        spend_df = spend_df[spend_df[col_name] != 'Unknown']
        spend_df = spend_df[spend_df[col_name] != 'Other']
    
    ## group spend data on column and find total revenue and puchases
    col_groups = spend_df.groupby(col_name).agg({'revenue': 'sum', 
                                                 'total_purchases': 'sum'})
    
    col_groups = col_groups[col_groups.revenue > 0]
    
    ## column of avg spend per group per transaction
    col_groups["avg_spend"] = col_groups.revenue/col_groups.total_purchases

    if sort_order != "index":
        col_groups = col_groups.sort_values(sort_order, ascending = False)
    
    ## percentage of revenue share and total purchases share for each group
    col_groups["pct_rev"] = 100*col_groups["revenue"]/col_groups["revenue"].sum()   
    col_groups["pct_total_purchases"] = 100*col_groups["total_purchases"]/col_groups["total_purchases"].sum()
    
    col_groups = col_groups.round(2)
    
    ## add count of customers in a group
    if include_segment_count:
        col_count = segment_count.groupby(col_name).size().reset_index().rename(columns = {0:"cust_count"})
        col_groups = pd.merge(col_groups, col_count, left_index=True, right_on=col_name)
        col_groups.set_index(col_name, inplace = True)
        
        col_groups["pct_cust_count"] = 100*col_groups["cust_count"]/col_groups["cust_count"].sum()
        
        col_groups["rev_per_customer"] = col_groups["revenue"]/col_groups["cust_count"]
    
    display(col_groups.style.background_gradient(cmap="Blues"))

In [8]:
def spending_category_stats_by_col(spend_df, 
                                   col_name, 
                                   transpose = True):
    """
    This function takes the spend data and generates top purchase categories for individual groups in a field
    We create a pivot table which depicts a cross tabulation of groups in a field with all spend categories:

    Parameters:
        spend_df: spend data (segment_revenue_and_count)
        col_name: the column to generate stats on (eg.: Age, Income, Gender etc.)
        
    """
    
    ## change data type to str  
    spend_df[col_name] = spend_df[col_name].astype(str)
    spend_df["PurchaseCategoryA"] = spend_df["PurchaseCategoryA"].astype(str)
    
    ## remove unknowns
    filtered_segment_revenue_and_count = spend_df[(spend_df[col_name] != 'Unknown') & (spend_df["PurchaseCategoryA"] != 'Unknown')]
    
    ## create pivot table 
    col_name_purchase_pivot = pd.pivot_table(filtered_segment_revenue_and_count, columns = col_name, index = "PurchaseCategoryA", aggfunc={"revenue":np.sum, "total_purchases":np.sum})
    
    ## add revenue share for Gender
    if col_name == "Gender":
        col_name_purchase_pivot["female/male rev"] = col_name_purchase_pivot.revenue.Female / col_name_purchase_pivot.revenue.Male
    
    ## add revenue share for Housing catagories
    if col_name == "Housing":
        col_name_purchase_pivot["House/Flat rev"] = col_name_purchase_pivot.revenue.House / col_name_purchase_pivot.revenue.Flat
    
    
    display(col_name_purchase_pivot.style.background_gradient(cmap="Blues"))
    
    return col_name_purchase_pivot

In [9]:
def spending_category_stats_by_col_transpose(spend_df, 
                                             col_name):
    
    """
    This function serves the same object as the spending_category_stats_by_col but inverts the pivot index and columns

    Parameters:
        spend_df: spend data (segment_revenue_and_count)
        col_name: the column to generate stats on (eg.: Age, Income, Gender etc.)
        
    """
    
    ## change data type to str  
    spend_df[col_name] = spend_df[col_name].astype(str)
    spend_df["PurchaseCategoryA"] = spend_df["PurchaseCategoryA"].astype(str)
    
    ## remove unknowns
    filtered_segment_revenue_and_count = spend_df[(spend_df[col_name] != 'Unknown') & (spend_df["PurchaseCategoryA"] != 'Unknown')]
    
    ## create pivot table 
    col_name_purchase_pivot = pd.pivot_table(filtered_segment_revenue_and_count, index = col_name, columns = "PurchaseCategoryA", aggfunc={"revenue":np.sum, "total_purchases":np.sum})

    display(col_name_purchase_pivot.style.background_gradient(cmap="Blues"))

In [10]:
def spending_category_stats_by_col_rev_per_cust(spending_category_pivot, 
                                                col_name):
    """
    This function manipulates the pivot table generated by spending_category_stats_by_col method
    and determines the revenue per customer for each field-group * spend category.
    
    Parameters:
        spending_category_pivot: pivot table genearted by spending_category_stats_by_col function
        col_name: the column to generate stats on (eg.: Age, Income, Gender etc.)
        
    """
    
    ## find count of customers in segment
    col_count = segment_count.groupby(col_name).size().reset_index().rename(columns = {0:"cust_count"})
    
    ## merge count with groups in the spending pivot 
    spending_category_pivot = pd.merge(spending_category_pivot.revenue.T, col_count, left_index=True, right_on=col_name)
    spending_category_pivot.set_index(col_name, inplace = True)
    
    ## the percentage of cystomers in a category
    spending_category_pivot["pct_cust_count"] = 100*spending_category_pivot["cust_count"]/spending_category_pivot["cust_count"].sum()
    
    ## revenue per customer in a category
    spending_category_pivot[spending_category_pivot.columns[:-2]] = spending_category_pivot.iloc[:, :-2]/np.array(spending_category_pivot[["cust_count"]])
    spending_category_pivot = spending_category_pivot.iloc[:, :-2]
    spending_category_pivot = spending_category_pivot.T
    
    display(spending_category_pivot.style.background_gradient(cmap="Blues"))

# EDA

## Highest revenue generating product categories

In [11]:
get_spend_stats_by_col(segment_revenue_and_count, "PurchaseCategoryA")

## The first five spend categories (Home, Garden, Beauty, Electronics and Spas & Country House) 
## account for over 60% of the total revenue.

## Home, Garden, Beauty and Electronics are also the top four areas of footfall for consumers. 
## If we don't consider "Other" category, the next most transacted category is Fashion 
## (which is another common cusumer spend in general)

## The average ticket size is highest in Beach Holidays and Wedding, which also makes sense.
## But both of them combined account for less than 1% revenue and transactions.

## Another high ticket size and revenue generating categories are UK City Breaks and European City Breaks.

## In fact, every category that has over 1% of total revenue and pct_total_purchases are all commonly transacted areas.

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases
PurchaseCategoryA,,,,,
Home,46606606.300000,698896,66.690000,23.010000,13.480000
Garden,31731150.900000,549247,57.770000,15.670000,10.600000
Beauty,18362390.620000,466295,39.380000,9.070000,9.000000
Electronics,17152439.750000,478333,35.860000,8.470000,9.230000
Spas & Country House,13089396.260000,93329,140.250000,6.460000,1.800000
UK City Breaks,12750782.250000,111729,114.120000,6.300000,2.160000
European City Breaks,9224984.610000,70656,130.560000,4.560000,1.360000
Restaurants & Bars,7657227.810000,241948,31.650000,3.780000,4.670000
Activities,6765654.630000,273882,24.700000,3.340000,5.280000


### Top sub-categories in Home Spending

In [12]:
get_spend_stats_by_col(segment_revenue_and_count, "PurchaseCategoryB", filter_col="PurchaseCategoryA", filter_value="Home")

## Beds & Mattresses primarily dominates Home spending. The most revenue and footfall comes from this sub-category.

## Bedding is another area many purchases take place, however, the average ticket size is small.

## Relatively less transactions take place to purchase Sofas & Futons, but this area has highest ticket size.

## Dining Furniture, Bedroom Furniture and Chairs & Stools & Beanbags are three more areas of high ticket size

## Canvas/Art/Accessories and Cleaning & Home Maintenance also receive high footfall for Home spending

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases
PurchaseCategoryB,,,,,
Beds & Mattresses,27158579.370000,163993,165.610000,58.270000,23.460000
Sofas & Futons,3523261.250000,8283,425.360000,7.560000,1.190000
Bedding,3375152.250000,149480,22.580000,7.240000,21.390000
Chairs & Stools & Beanbags,2295070.560000,28598,80.250000,4.920000,4.090000
Bedroom Furniture,1788195.510000,18731,95.470000,3.840000,2.680000
Cleaning & Home Maintenance,1775733.410000,60601,29.300000,3.810000,8.670000
Storage Solutions,1268211.230000,30797,41.180000,2.720000,4.410000
Canvas/Art/Accessories,870657.230000,74581,11.670000,1.870000,10.670000
Soft Furnishings,727407.250000,36309,20.030000,1.560000,5.200000


### Top sub-categories in Garden Spending

In [13]:
get_spend_stats_by_col(segment_revenue_and_count, "PurchaseCategoryB", filter_col="PurchaseCategoryA", filter_value="Garden")

## Garden Furniture provides the highest revenue in Garden Spending, over 50% of total revenue

## Plants & Flowers has the highest footfall though, which is also explained by its avg ticket size,
## indicating more affordability

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases
PurchaseCategoryB,,,,,
Garden Furniture,17029006.970000,68573,248.330000,53.670000,12.480000
Garden Leisure,5117384.320000,118608,43.150000,16.130000,21.590000
Plants & Flowers,3782637.120000,159318,23.740000,11.920000,29.010000
Storage Solutions,1688849.620000,31335,53.900000,5.320000,5.710000
Garden Lighting,1490336.850000,74345,20.050000,4.700000,13.540000
Garden Tools,1365630.150000,65641,20.800000,4.300000,11.950000
BBQ & Picnic,1017973.720000,19862,51.250000,3.210000,3.620000
Indoor Plants,205457.240000,9681,21.220000,0.650000,1.760000
Other,33874.900000,1884,17.980000,0.110000,0.340000


### Top sub-categories in Electronics Spending

In [14]:
get_spend_stats_by_col(segment_revenue_and_count, "PurchaseCategoryB", remove_unknowns=True, filter_col="PurchaseCategoryA", filter_value="Electronics")

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases
PurchaseCategoryB,,,,,
Accessories,4229225.900000,182606,23.160000,24.670000,38.180000
Computing,3271715.570000,32162,101.730000,19.090000,6.720000
Tablets,2651540.720000,29797,88.990000,15.470000,6.230000
Camera & Photo,1357962.200000,42034,32.310000,7.920000,8.790000
Software,1253550.180000,41274,30.370000,7.310000,8.630000
Headphones,1033343.920000,48291,21.400000,6.030000,10.100000
White Goods,946423.900000,27420,34.520000,5.520000,5.730000
Gaming,735983.270000,11125,66.160000,4.290000,2.330000
Mobile Phones,633218.940000,4665,135.740000,3.690000,0.980000


## Does Age Influence Revenue and Total Transactions?

### Without unknown

In [15]:
get_spend_stats_by_col(segment_revenue_and_count, "Age", remove_unknowns=True, sort_order="index", include_segment_count = True)

## 1) average revenue per transaction is roughly the same in each age group.
## This shows that customers are equally engaged irrespective of the age group.

## 2) The total revenue, as well as number of transactions are the highest for 45-55 (not considering unknown)
## These are the most actively engaged customers

## 3) Despite point 2), roughly 65% of the total revenue is coming from 25-65+ group.

## 4) Overall, there is no major indication that Age has any influence on generating revenue except for kids
## and all age-groups are equally important for revenue.

## 5) The average revenue per customer of Age 18-14 is also the lowest

## 6) Although 45-55 age group has the highest revenue, the rev_per_customer is the highest for 65+, which also has
## the least number of customers

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases,cust_count,pct_cust_count,rev_per_customer
Age,,,,,,,,
18-24,5791287.280000,145401,39.830000,4.170000,4.090000,714,16.331199,8111.046611
25-34,27680683.210000,707211,39.140000,19.940000,19.880000,833,19.053065,33230.111897
35-44,28377308.780000,727407,39.010000,20.440000,20.450000,818,18.709973,34691.086528
45-54,29592209.310000,747204,39.600000,21.320000,21.010000,748,17.108875,39561.777152
55-64,23644179.320000,604612,39.110000,17.030000,17.000000,663,15.164684,35662.412247
65+,23729479.170000,624715,37.980000,17.090000,17.570000,596,13.632205,39814.562366


### Top Spending Categories by Age

In [16]:
col_groups = spending_category_stats_by_col(segment_revenue_and_count, "Age")

## Irrespective of age group, Home spending remains the dominating category for revenue and total purchases

## However, home spending typically decreases with age

## Instead, there is a transitional shift in spending towards Garden.

## Beauty spending also decreases with increasing Age

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [17]:
spending_category_stats_by_col_transpose(segment_revenue_and_count, "Age")

## With Age, Spending decreases in almost all categories

## It reaches the peak at 35-44 and 45-54 age grooups

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories for different Age groups in terms of Revenue per customer

In [18]:
spending_category_stats_by_col_rev_per_cust(col_groups, "Age")

## Irrespective of age group, Home spending generated the highest rev per customer

## Avg rev increases in Garden activities for people in higher age groupds

<IPython.core.display.Javascript object>

Age,18-24,25-34,35-44,45-54,55-64,65+
Activities,279.526979,1138.526501,1261.911250,1360.118702,1134.576352,1257.857999
Adult,22.848435,107.639216,101.996966,109.658377,89.217219,109.078373
Beach Holidays,59.703494,277.406110,302.951220,295.089795,238.954940,253.573988
Beauty,800.637488,3274.117184,3524.004841,3655.740789,2970.950424,3060.628744
Children and Baby,212.749525,983.424481,994.904175,981.928364,929.145072,1046.248206
Electronics,702.536698,2985.233515,3122.925787,3187.546926,2811.689699,3189.089926
Entertainment,142.608047,591.639142,685.471146,727.326886,624.947529,664.921926
European City Breaks,465.539380,1618.351292,1516.529236,1786.338748,1528.299031,1524.090495
Fashion,216.365720,874.029247,939.502652,1116.863628,1043.138133,1248.742135
Fitness,181.573529,732.768427,773.988846,877.623706,759.629164,891.919714


## What is the Gender-wise share on Revenue and Total Transactions

### Without Unknowns

In [19]:
get_spend_stats_by_col(segment_revenue_and_count, "Gender", remove_unknowns=True, sort_order="index", include_segment_count = True)

## Females spend less per transaction but they have significantly higher revenue share and footfall

## The difference between in the total female and male customers is just 200. However, the difference in revenue
## generated is dispropotionately higher. This requirs further investigation into what categories female
## tend to spend the most

## The average revenue for females is roughly 2.5 times that of Males

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases,cust_count,pct_cust_count,rev_per_customer
Gender,,,,,,,,
Female,97184006.500000,2623816,37.040000,74.220000,77.130000,1783,52.986627,54505.892597
Male,33748503.750000,777886,43.380000,25.780000,22.870000,1582,47.013373,21332.808944


### Top Spending Categories by Gender

In [20]:
col_groups = spending_category_stats_by_col(segment_revenue_and_count, "Gender")

## Males spend most towards Electronics, Garden and Home

## Females spend most towards Home, over 3 times as such as Males

## Other top categories are Beauty, Garden, Fashion, Electronics.

## Females spend 7.5 times more in weddings than Males
## Females also have a stronger liking for Pets than Males
## They spend more in Learning than Males

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [21]:
spending_category_stats_by_col_transpose(segment_revenue_and_count, "Gender")

## For all categories except Adult, female spending and footfall is higher

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories for different Genders in terms of Revenue per customer

In [22]:
spending_category_stats_by_col_rev_per_cust(col_groups, "Gender")

## Irrespective of the Gender, Home and Garden spending generated the highest rev per customer

## Females spend much more in Beauty than Males

<IPython.core.display.Javascript object>

Gender,Female,Male
Activities,1903.556073,568.398582
Adult,111.564977,125.906674
Beach Holidays,400.185057,191.922706
Beauty,5251.693171,1365.465825
Children and Baby,1611.762068,442.529083
Electronics,4054.834287,2424.586892
Entertainment,1031.862377,319.139690
European City Breaks,2224.135664,995.105909
Fashion,1779.273200,424.778497
Fitness,1124.937113,556.096021


## Does Income translate into Revenue and Total Transactions?

### Without Unknowns

In [23]:
get_spend_stats_by_col(segment_revenue_and_count, "Income",remove_unknowns=True, sort_order="index", include_segment_count = True)

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases,cust_count,pct_cust_count,rev_per_customer
Income,,,,,,,,
High,29642422.430000,705471,42.020000,15.950000,14.830000,1343,26.567755,22071.796299
Low,84461846.440000,2257105,37.420000,45.440000,47.450000,1403,27.754698,60200.888411
Middle,63574250.300000,1601121,39.710000,34.200000,33.660000,1369,27.082097,46438.458948
VeryHigh,8209065.510000,193197,42.490000,4.420000,4.060000,940,18.595450,8733.048415


In [24]:
## Low and middle income categories collectively generate 80% of the revenue

## The fraction (or total number) of VeryHigh income category inviduals isn't that far off from other categories.
## Yet, the revenue, total purchases and total rev per customers is significantly lower

## Low income category have the highest revenue per customer, followed by Mid.

### Top Spending Categories by Income

In [25]:
col_groups = spending_category_stats_by_col(segment_revenue_and_count, "Income")
## Males spend most towards Electronics, Garden and Home

## Females spend most towards Home, over 3 times as such as Males

## Other top categories are Beauty, Garden, Fashion, Electronics.

## Females spend 7.5 times more in weddings than Males
## Females also have a stronger liking for Pets than Males
## They spend more in Learning than Males

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
spending_category_stats_by_col_transpose(segment_revenue_and_count, "Income")

## For all categories except Adult, female spending and footfall is higher

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories for different Income groups in terms of Revenue per customer

In [27]:
spending_category_stats_by_col_rev_per_cust(col_groups, "Income")

## rev per customer is highest for VeryHigh income groups

## Garden and Home is a top choice for all groups

<IPython.core.display.Javascript object>

Income,High,Low,Middle,VeryHigh
Activities,813.290886,1916.774178,1611.276586,306.164834
Adult,44.415428,202.776580,119.157160,16.330498
Beach Holidays,191.006072,439.005991,343.767732,75.629743
Beauty,2046.661684,5444.403974,4157.995174,784.635362
Children and Baby,513.398700,1800.383457,1201.261952,184.866260
Electronics,1507.965865,5703.998372,3642.244428,567.674451
Entertainment,448.018584,1003.745556,852.822086,181.081248
European City Breaks,1106.621778,2450.721575,2097.473199,461.625803
Fashion,538.704629,1838.429341,1307.771171,200.981529
Fitness,483.032487,1338.264209,1022.034911,186.525137


## Children-wise Revenue and Total Transactions

### Without Unknowns

In [28]:
get_spend_stats_by_col(segment_revenue_and_count, "Children", remove_unknowns=True, sort_order="index", include_segment_count = True)

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases,cust_count,pct_cust_count,rev_per_customer
Children,,,,,,,,
Baby,5417206.010000,136735,39.620000,2.910000,2.870000,923,18.259149,5869.128938
None,113640747.100000,2928895,38.800000,61.130000,61.570000,1166,23.066271,97462.047256
Preschool,6903361.320000,175404,39.360000,3.710000,3.690000,908,17.962413,7602.820837
Teenager,51133677.440000,1288502,39.680000,27.510000,27.090000,1119,22.136499,45695.869026
Youth,8792592.800000,227358,38.670000,4.730000,4.780000,939,18.575668,9363.783600


In [29]:
## The highest revenue is generated by individuals with no children. It would be good to consider
## relationship status together to get more insights

In [30]:
filtered_segment_revenue_and_count = segment_revenue_and_count[segment_revenue_and_count.Relationship != 'Unknown']
filtered_segment_revenue_and_count = filtered_segment_revenue_and_count[filtered_segment_revenue_and_count.Children != 'Unknown']
filtered_segment_revenue_and_count["Children"] = filtered_segment_revenue_and_count["Children"].astype(str)
filtered_segment_revenue_and_count["Relationship"] = filtered_segment_revenue_and_count["Relationship"].astype(str)


Relationship_child_pivot = pd.pivot_table(filtered_segment_revenue_and_count, index = "Children", columns = "Relationship", aggfunc={"revenue":np.sum, "total_purchases":np.sum})
display(Relationship_child_pivot.style.background_gradient(cmap="Blues"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [31]:
## Irrespective of the Relationship status, folks spend the most and transact the most who have no children

In [32]:
Relationship_child_pivot_transpose = pd.pivot_table(filtered_segment_revenue_and_count, columns = "Children", index = "Relationship", aggfunc={"revenue":np.sum, "total_purchases":np.sum})
display(Relationship_child_pivot_transpose.style.background_gradient(cmap="Blues"))

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories by Children

In [33]:
col_groups = spending_category_stats_by_col(segment_revenue_and_count, "Children")

## 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [34]:
spending_category_stats_by_col_transpose(segment_revenue_and_count, "Children")

## 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories for different Children groups in terms of Revenue per customer

In [35]:
spending_category_stats_by_col_rev_per_cust(col_groups, "Children")



<IPython.core.display.Javascript object>

Children,Baby,None,Preschool,Teenager,Youth
Activities,197.303260,3140.907328,270.174273,1648.561273,362.778769
Adult,20.689773,271.559805,24.803995,123.654842,27.683804
Beach Holidays,58.169713,707.455031,84.213661,358.303957,61.943770
Beauty,630.560413,8686.114673,761.758385,4095.554429,886.473974
Children and Baby,162.536774,2557.167966,229.106913,1249.109528,315.519076
Electronics,530.504639,7929.831662,690.224210,3893.713183,881.923690
Entertainment,105.319376,1753.956503,142.726678,814.502804,175.001334
European City Breaks,335.330690,4207.999667,388.284157,2037.082590,406.633878
Fashion,145.893406,2829.203741,192.774842,1284.663944,251.297514
Fitness,125.762091,2149.598873,167.829992,1010.857070,207.398668


## Revenue and Total Transactions Split by Housing

### Without Unknowns

In [36]:
get_spend_stats_by_col(segment_revenue_and_count, "Housing", remove_unknowns=True, sort_order="index", include_segment_count = True)

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases,cust_count,pct_cust_count,rev_per_customer
Housing,,,,,,,,
Flat,28799436.540000,757598,38.010000,15.090000,15.490000,1853,47.598253,15542.059655
House,162017837.220000,4133337,39.200000,84.910000,84.510000,2040,52.401747,79420.508441


### Top Spending Categories by Housing

In [37]:
col_groups = spending_category_stats_by_col(segment_revenue_and_count, "Housing")

## Irrespective of House or Flat, highest spending is in Home

## Moreover, for folks living in House, Garden spending is significantly higher

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [38]:
spending_category_stats_by_col_transpose(segment_revenue_and_count, "Housing")

## Overall, people living in house spend more than those living in a flat

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories for different Housing groups in terms of Revenue per customer

In [39]:
spending_category_stats_by_col_rev_per_cust(col_groups, "Housing")



<IPython.core.display.Javascript object>

Housing,Flat,House
Activities,440.331256,2726.947394
Adult,60.271754,209.843243
Beach Holidays,162.942862,561.405091
Beauty,1810.303252,6807.264237
Children and Baby,383.525548,2198.665331
Electronics,1506.828458,6494.215107
Entertainment,314.886586,1386.933889
European City Breaks,724.071063,3457.568015
Fashion,436.977083,2270.943366
Fitness,339.591065,1767.554532


## Relationship-wise Revenue and Total Transactions

### Without Unknowns

In [40]:
get_spend_stats_by_col(segment_revenue_and_count, "Relationship", remove_unknowns=True, sort_order="index", include_segment_count = True)

## Married people not only generate the most revenue per customer, but also total revenue, transactions, avg spen etc.

## Singles are the next in terms of revenue

,revenue,total_purchases,avg_spend,pct_rev,pct_total_purchases,cust_count,pct_cust_count,rev_per_customer
Relationship,,,,,,,,
Cohabiting,14333791.200000,375620,38.160000,8.910000,9.140000,949,23.022804,15104.100316
Married,75669961.480000,1886741,40.110000,47.060000,45.900000,1107,26.855895,68355.882096
Separated,27652722.530000,732646,37.740000,17.200000,17.820000,1021,24.769529,27083.959383
Single,43144698.690000,1115953,38.660000,26.830000,27.150000,1045,25.351771,41286.793005


### Top Spending Categories by Relationship

In [41]:
col_groups = spending_category_stats_by_col(segment_revenue_and_count, "Relationship")

## Irrespective of relationship, people spend the most in Home.

## Married people are typically interested in Garden and Spas & Country House than others 

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [42]:
spending_category_stats_by_col_transpose(segment_revenue_and_count, "Relationship")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

### Top Spending Categories for different Relationship groups in terms of Revenue per customer

In [43]:
spending_category_stats_by_col_rev_per_cust(col_groups, "Relationship")



<IPython.core.display.Javascript object>

Relationship,Cohabiting,Married,Separated,Single
Activities,542.033500,2430.920664,829.679889,1309.326295
Adult,45.886731,159.537239,86.654602,136.250281
Beach Holidays,108.822144,443.688582,196.555071,424.014537
Beauty,1395.329482,5682.249709,2348.984934,4443.788011
Children and Baby,470.317739,1746.428428,756.462513,1116.184966
Electronics,1304.827146,5131.641570,2411.760589,3871.863791
Entertainment,260.922105,1220.197269,464.502332,789.852235
European City Breaks,644.465114,3023.908751,1122.251424,1932.876823
Fashion,433.899849,1907.412295,833.037491,1121.445950
Fitness,332.338706,1491.996180,607.481252,900.883142


# Personalisation

## Workflow

In [51]:
## Overall, the above EDA, for each customer category (Age, Income, Children, Housing, Gender and Relationship)
## indicates that for each segment, there is some product category where people spend the most
## and generate the most revenue per customer.

## As there is no time series element in the given data, we cannot determine periodic patterns in data. But given
## that we have categorical information which creates a segment, we can determine the top purchases and revenue
## generating products for each individual groups.

## revenue per customer is a good metric to select as it aligns pretty well with most businesses.

## For personalisation, the following recommendation workflow may work:

## Define a function which will be parameterized with the 6 categories of a segment.

## Firstly, we shall determine the products that are most popular for that whole group collectively. 
## This is a direct recommendation from the spend history so 
## will consider total_purchases as the metric to detemine the popular category and return top {k} products. 

## Secondly, as recommending new products is aimed to increase revenue, we will consider each of the 6 categories 
## individually and determine the top {k} products with the highest revenue per customer. These are the products that
## will be recommended to the customers as new suggestions.
## Here, we shall only recommend those products which aren't the top purchases already of that group

In [45]:
segment_revenue_and_count["Age"] = segment_revenue_and_count["Age"].astype(str)
segment_revenue_and_count["Gender"] = segment_revenue_and_count["Gender"].astype(str)
segment_revenue_and_count["Income"] = segment_revenue_and_count["Income"].astype(str)
segment_revenue_and_count["Children"] = segment_revenue_and_count["Children"].astype(str)
segment_revenue_and_count["Housing"] = segment_revenue_and_count["Housing"].astype(str)
segment_revenue_and_count["Relationship"] = segment_revenue_and_count["Relationship"].astype(str)
segment_revenue_and_count["PurchaseCategoryA"] = segment_revenue_and_count["PurchaseCategoryA"].astype(str)
segment_revenue_and_count["PurchaseCategoryB"] = segment_revenue_and_count["PurchaseCategoryB"].astype(str)
segment_revenue_and_count["PurchaseCategory"] = segment_revenue_and_count['PurchaseCategoryA'].str.cat(segment_revenue_and_count['PurchaseCategoryB'], sep='|').values

segment_revenue_and_count.head()

,Age,Gender,Income,Children,Housing,Relationship,PurchaseCategoryA,PurchaseCategoryB,revenue,total_purchases,avg_spend,PurchaseCategory
0,35-44,Male,Middle,Youth,Flat,Married,Home,Beds & Mattresses,733.450,6,122.241667,Home|Beds & Mattresses
1,Unknown,Female,Low,Youth,House,Unknown,Home,Beds & Mattresses,10745.972,67,160.387642,Home|Beds & Mattresses
2,45-54,Unknown,VeryHigh,None,House,Separated,Home,Beds & Mattresses,2204.830,16,137.801875,Home|Beds & Mattresses
3,25-34,Unknown,Middle,Teenager,House,Married,Home,Beds & Mattresses,25627.180,145,176.739172,Home|Beds & Mattresses
4,35-44,Unknown,Middle,None,Flat,Unknown,Home,Beds & Mattresses,3405.240,21,162.154286,Home|Beds & Mattresses


## Algorithm

In [46]:
def get_most_popular_product_for_full_category(spend_df, age, gender, income, children, housing, relationship, count):
    
    """
    This function accepts the whole segment and returns the most frequently purchased items
    
    Parameters:
    
        1. spend_df: spend data
        2. (age, gender, ... , relationship): segment details
        3. count: number of top purchases to return
    
    """
    
    ## select the segment
    spend_df = spend_df[spend_df.Age == age]
    spend_df = spend_df[spend_df.Gender == gender]
    spend_df = spend_df[spend_df.Income == income]
    spend_df = spend_df[spend_df.Children == children]
    spend_df = spend_df[spend_df.Housing == housing]
    spend_df = spend_df[spend_df.Relationship == relationship]
    
    ## find the most purchased product
    spend_df = spend_df.nlargest(count, "total_purchases", keep = "all")
    spend_df = spend_df.sort_values(["total_purchases", "avg_spend"], ascending = False).head(count)
    
    top_products = list(spend_df.PurchaseCategory.values)
        
    return top_products

In [47]:
def get_most_popular_product_for_all_cols(spend_df, age, gender, income, children, housing, relationship, count, selected_products):
    
    """
    This function, in contrast to the above function, filter data on just one field at a time and finds
    the most purchased product in the whole selected segment.
    
    For example, if the arguments specified were: "18-24", "Female", "VeryHigh", "None", "House", "Separated",
    first select the whole "18-24" segment, find the most popular product
    the select the Femal segment, find the most popular product and so on..
    
    Parameters:
    
        1. spend_df: spend data
        2. (age, gender, ... , relationship): segment details
        3. count: number of top purchases to return
        4. selected_products: products recommended in the previous step(s)
    
    """
    
    params = (age, gender, income, children, housing, relationship)

    ## remove selected_products to show new products
    spend_df = spend_df[~spend_df.PurchaseCategory.isin(selected_products)]
    top_products_filtered = pd.DataFrame()
    
    
    ## iterate over each field and find the most purchased items
    for idx, col_name in enumerate(['Age', 'Gender', 'Income', 'Children', 'Housing', 'Relationship']):
    
        df_filtered = spend_df[spend_df[col_name] == params[idx]]

        df_filtered = df_filtered.nlargest(count, "total_purchases", keep = "all")
        df_filtered = df_filtered.sort_values(["total_purchases", "avg_spend"], ascending = False).head(count)
        
        ## gather all products from each field
        top_products_filtered = pd.concat((top_products_filtered, df_filtered))
    
    ## find the most purchased products from the gathered products
    top_products_filtered = top_products_filtered.sort_values(["total_purchases", "avg_spend"], ascending = False)
    top_products_filtered = top_products_filtered.drop_duplicates(subset = ["PurchaseCategory"])
    top_products_filtered = top_products_filtered.head(count)
    
    top_products = list(top_products_filtered.PurchaseCategory.values)
    
    return top_products

In [48]:
def get_top_rev_per_customer_product_for_col(spend_df, age, gender, income, children, housing, relationship, count, selected_products):
    """
    This function finds the products that generate the most revenue per customer.
    The idea is similar to the above function get_most_popular_product_for_all_cols.
    
    For example, if the arguments specified were: "18-24", "Female", "VeryHigh", "None", "House", "Separated",
    first select the whole "18-24" segment, find the products which generate most revenue per customer
    the select the Femal segment, find the products which generate most revenue per customer and so on..
    
    Parameters:
    
        1. spend_df: spend data
        2. (age, gender, ... , relationship): segment details
        3. count: number of top purchases to return
        4. selected_products: products recommended in the previous step(s)
    
    """
    
    spend_df = spend_df[~spend_df.PurchaseCategory.isin(selected_products)]
    top_products_filtered = pd.DataFrame()
    
    params = (age, gender, income, children, housing, relationship)
    
    new_products = []
    
    for idx, col_name in enumerate(['Age', 'Gender', 'Income', 'Children', 'Housing', 'Relationship']):
        
        df_filtered = spend_df[spend_df[col_name] == params[idx]]
        df_filtered = df_filtered.groupby([col_name, "PurchaseCategory"]).revenue.sum().reset_index()
        df_filtered = df_filtered[df_filtered.revenue > 0]
        
        df_filtered["revenue_per_customer"] = df_filtered.revenue/segment_count[segment_count[col_name] == params[idx]].shape[0]
        df_filtered = df_filtered.nlargest(count, "revenue_per_customer", keep = "all")
        df_filtered = df_filtered.sort_values(["revenue_per_customer", "revenue"], ascending = False).head(count)
        
        top_products_filtered = pd.concat((top_products_filtered, df_filtered))
        
    top_products_filtered = top_products_filtered.sort_values(["revenue_per_customer", "revenue"], ascending = False)
    top_products_filtered = top_products_filtered.drop_duplicates(subset = ["PurchaseCategory"])
    
    top_products_filtered = top_products_filtered.head(count)
    
    top_products = list(top_products_filtered.PurchaseCategory.values)
    
    return top_products

In [49]:
def recommend_product_for_category(spend_df, age, gender, income, children, housing, relationship, count):
    
    params = (age, gender, income, children, housing, relationship)
    
    top_products = []
    
    ## get most purchased product for full segment
    top_products += get_most_popular_product_for_full_category(spend_df, age, gender, income, children, housing, relationship, count)
    
    ## get most purchased product by considering individual fields in segment
    top_products += get_most_popular_product_for_all_cols(spend_df, age, gender, income, children, housing, relationship, count, top_products)
    
    ## get products with most revenue per customer by considering individual fields in segment
    top_products += get_top_rev_per_customer_product_for_col(spend_df, age, gender, income, children, housing, relationship, count, top_products)
    
    return list(top_products)

In [50]:
recommend_product_for_category(segment_revenue_and_count, "18-24", "Female", "VeryHigh", "None", "House", "Separated", 4)

['Fitness|Sport Equipment',
 'Other|Wowcher Gift Card',
 'Beauty|Spa',
 'Fashion|Cosy-wear & Nightwear',
 'Children and Baby|Toys',
 'Garden|Plants & Flowers',
 'Home|Bedding',
 'Home|Beds & Mattresses',
 'Garden|Garden Furniture',
 'Spas & Country House|Country House Hotels',
 'Garden|Garden Leisure',
 'European City Breaks|Other Cities']

## Q) Please also describe any additional data points you would like to utilise with a personalisation algorithm

1) Purchases are frequently driven by cyclical patterns. However, in this data, we don't have any temporal information.
This makes it hard toidentify patterns in customer behavior and preferences over time and enable the algorithm to make more accurate predictions.

2) Customer feedback is also missing, which is crucial. Feedback from customers such as ratings can be used to generate recommendations for other customers who have similar personal information like Age, Gender etc.

3) Demographical data, if available, can also be utilised in this problem. The idea is that customers in a particular region may have some specific product preferences, which can be recommended to other customers.
